In [2]:
import psycopg2
def get_db_connection():
    """데이터베이스 연결을 반환합니다."""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="mydb",
            user="admin",
            password="admin123"
        )
        return conn
    except psycopg2.Error as e:
        raise

In [3]:
import psycopg2

def get_all_tables():
    """데이터베이스의 모든 테이블 이름을 조회합니다."""
    conn = None
    try:
        # DB 연결
        conn =get_db_connection()
        cur = conn.cursor()

        # 모든 테이블 이름 조회 (시스템 테이블 제외)
        query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' 
        ORDER BY table_name;
        """
        
        cur.execute(query)
        tables = cur.fetchall()

        print("--- 데이터베이스의 모든 테이블 ---")
        for table in tables:
            print(f"테이블명: {table[0]}")

        return [table[0] for table in tables]

    except (Exception, psycopg2.DatabaseError) as error:
        print("오류 발생:", error)
        return []
    finally:
        if conn is not None:
            cur.close()
            conn.close()

# 사용 예시
get_all_tables()

--- 데이터베이스의 모든 테이블 ---
테이블명: builder
테이블명: products


['builder', 'products']

In [18]:
# def drop_table(table_name):
#     """지정된 테이블을 삭제합니다."""
#     conn = None
#     try:
#         conn = get_db_connection()
#         cursor = conn.cursor()
        
#         # 테이블 삭제 SQL 실행
#         drop_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
#         cursor.execute(drop_query)
#         conn.commit()
        
#         print(f"테이블 '{table_name}'이(가) 성공적으로 삭제되었습니다.")
        
#     except psycopg2.Error as e:
#         if conn:
#             conn.rollback()
#         print(f"테이블 삭제 오류: {e}")
#         raise
#     finally:
#         if conn:
#             cursor.close()
#             conn.close()
# drop_table('vessel')  # 예시 테이블명

In [19]:
import psycopg2
from psycopg2 import sql

def create_table():
    """PostgreSQL 데이터베이스에 새로운 테이블을 생성합니다."""
    conn = None
    try:
        # 데이터베이스 연결
        conn = psycopg2.connect(
            host="localhost",
            database="mydb",
            user="admin",
            password="admin123"
        )

        # 커서(cursor) 생성
        cur = conn.cursor()

        # 실행할 SQL 쿼리 (테이블 생성)
        # IF NOT EXISTS를 사용하여 테이블이 이미 존재하면 에러를 방지합니다.
        create_table_query = """
        CREATE TABLE IF NOT EXISTS vessel (
            id SERIAL PRIMARY KEY,
            row VARCHAR(50) NOT NULL,
            fleet_type VARCHAR(50) NOT NULL,
            type VARCHAR(50) NOT NULL,
            sfc_vessel_types VARCHAR(50) NOT NULL,
            ofc_vessel_types VARCHAR(50) NOT NULL,
            special_value VARCHAR(50) NOT NULL,
            special_unit VARCHAR(50) NOT NULL,
            dwt VARCHAR(50) NOT NULL,
            gt VARCHAR(50) NOT NULL,
            cgt VARCHAR(50) NOT NULL,
            loa_unit_m VARCHAR(50) NOT NULL,
            beam_mid_unit_m VARCHAR(50) NOT NULL,
            draft_unit_m VARCHAR(50) NOT NULL,
            speed_unit_knots VARCHAR(50) NOT NULL,
            consumption_unit_tpd_tons_per_day VARCHAR(50) NOT NULL,
            contract_date VARCHAR(50) NOT NULL,
            contract_year VARCHAR(50) NOT NULL,
            built_date VARCHAR(50) NOT NULL,
            built_year VARCHAR(10) NOT NULL,
            original_builder VARCHAR(50) NOT NULL,
            original_builder_country_region VARCHAR(50) NOT NULL,
            original_builder_group VARCHAR(50) NOT NULL,
            builder_group_country_region VARCHAR(50) NOT NULL,
            first_company VARCHAR(50) NOT NULL,
            first_company_nationality VARCHAR(50) NOT NULL,
            group_company VARCHAR(50) NOT NULL,
            group_company_nationality_region VARCHAR(50) NOT NULL,
            designer VARCHAR(50) NOT NULL,
            status VARCHAR(50) NOT NULL,
            price_unit_nb VARCHAR(50) NOT NULL,
            currency_unit_nb VARCHAR(50) NOT NULL,
            power_type VARCHAR(50) NOT NULL,
            main_engine_fuel_type VARCHAR(50) NOT NULL,
            fuel_category VARCHAR(50) NOT NULL,
            main_engine_no1_manufacturer_company VARCHAR(50) NOT NULL,
            main_engine_detail VARCHAR(1012) NOT NULL,
            auxiliary_engine_detail VARCHAR(1012) NOT NULL,
            emergency_engine_detail VARCHAR(1012) NOT NULL,
            power_generation_equipment_summary VARCHAR(1012) NOT NULL,
            engine_room_equipment_summary VARCHAR(1012) NOT NULL,
            boiler_detail VARCHAR(1012) NOT NULL,
            environmental_detail VARCHAR(1012) NOT NULL,
            est_detail_energy_saving_technologies VARCHAR(1012) NOT NULL,
            flag_state VARCHAR(50) NOT NULL,
            hull_no VARCHAR(50) NOT NULL,
            old_type VARCHAR(50) NOT NULL,
            remark VARCHAR(200) NOT NULL,
            created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
        );
        """

        # SQL 쿼리 실행
        cur.execute(create_table_query)

        # 변경사항을 데이터베이스에 커밋(commit)
        conn.commit()

        print("'vessel' 테이블이 성공적으로 생성되었습니다.")

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        # 연결 종료
        if conn is not None:
            cur.close()
            conn.close()
            print("PostgreSQL connection is closed.")

create_table()

'vessel' 테이블이 성공적으로 생성되었습니다.
PostgreSQL connection is closed.


In [20]:
import pandas as pd
def reform_excel_data(path:str, sheet_name:str):
    """엑셀 데이터를 재구성합니다."""

    df = pd.read_excel(path, sheet_name=sheet_name, dtype=str)
    df.fillna('', inplace=True)
    # NaN 값을 명시적으로 빈 문자열로 변환
    df = df.where(pd.notnull(df), '')
    reformed_result = [tuple(row) for row in df.values]
    return reformed_result

result = reform_excel_data('../data/dev_vessel.xlsx', 'Vessel')
print(len(result[0]))

def chunked_data(data: list, chunk_size: int):
    """데이터를 청크 단위로 나눕니다."""
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

47


In [21]:
from psycopg2.extras import execute_batch
def insert_multiple_products():
    """products 테이블에 여러 상품 데이터를 한 번에 삽입합니다."""
    conn = None
    cur = None
    
    try:
        # 데이터베이스 연결
        conn = get_db_connection()
        cur = conn.cursor()

        # SQL 쿼리
        sql = """INSERT INTO vessel 
        (row, fleet_type, type, sfc_vessel_types, ofc_vessel_types, special_value, special_unit, dwt, gt, cgt, loa_unit_m, beam_mid_unit_m,
        draft_unit_m, speed_unit_knots, consumption_unit_tpd_tons_per_day, contract_date, contract_year, built_date, built_year,
        original_builder, original_builder_country_region, original_builder_group, builder_group_country_region, first_company, 
        first_company_nationality, group_company, group_company_nationality_region, designer, status, price_unit_nb, currency_unit_nb,
        power_type, main_engine_fuel_type, fuel_category, main_engine_no1_manufacturer_company, main_engine_detail, auxiliary_engine_detail,
        emergency_engine_detail, power_generation_equipment_summary, engine_room_equipment_summary, boiler_detail, environmental_detail,
        est_detail_energy_saving_technologies, flag_state, hull_no, old_type, remark)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s)"""

        # 엑셀 데이터 로드
        reformed_data = reform_excel_data('../data/dev_vessel.xlsx', 'Vessel')
        
        # 청크 크기 설정 (한 번에 삽입할 행 수)
        chunk_size = 100  # 필요에 따라 조정
        
        total_inserted = 0
        
        for i, chunk in enumerate(chunked_data(reformed_data, chunk_size)):
            print(f">>> 청크 데이터 개수: {len(chunk)}")
            print(len(chunk[0]))
            print(chunk[0])
            try:
                # execute_batch를 사용하여 배치 삽입 (성능 향상)
                execute_batch(cur, sql, chunk)
                conn.commit()
                
                total_inserted += len(chunk)
                print(f"청크 {i+1} - {len(chunk)}개의 데이터 삽입 완료 (총 {total_inserted}개)")
                
            except Exception as chunk_error:
                print(f"청크 {i+1} 삽입 중 오류: {chunk_error}")
                conn.rollback()
                # 실패한 청크의 데이터 로깅 (디버깅용)
                print(f"실패한 청크 데이터: {len(chunk)} 개")  # 실패한 청크 개수
                print(f"실패한 청크 데이터: {chunk[:1]}")  # 처음 1개 행만 로깅
                
                continue  # 다음 청크로 계속 진행

        print(f"모든 데이터 삽입 완료: 총 {total_inserted}개의 행")

    except Exception as error:
        print(f"전체 처리 중 오류 발생: {error}")
        if conn:
            conn.rollback()
    finally:
        # 리소스 정리
        if cur:
            cur.close()
        if conn:
            conn.close()
            print("데이터베이스 연결 종료")

insert_multiple_products()

>>> 청크 데이터 개수: 10
47
('R000001', 'Containerships', 'Fully Cellular Container', 'Intermediate 3-7,999 TEU', '-', '4500', 'TEU', '55000', '47529', '28779.64955', '-', '-', '-', '-', '-', '2025-07-18 00:00:00', '2025', '2028-04-01 00:00:00', '2028', 'Taizhou Sanfu', 'China P.R.', 'Taizhou Sanfu', 'China P.R.', 'MPC Container Ships', 'Cyprus', 'Castor Maritime', 'Cyprus', '-', 'On Order', '57', '$m', 'Diesel 2-Stroke', 'VLS IFO', 'Diesel', 'Unknown', '1 x Diesel - Everllence B. & W. 6G70ME-C10.5 - 2-stroke 6-cyl. 700mm x3256mm bore/stroke 18,600mkW total at 78rpm', '3 x Aux. Diesel Gen. - 4-stroke  driving 3 x AC generator(s)', '1 x Emergency Diesel Gen. - 4-stroke  driving 1 x AC generator(s)', '-', '1 x Screw Shaft', '-', '3 x Selective Catalytic Reduction - NOx(Low Pressure) - Unknown, 1 x BWTS - Ballast Water Treatment System', '-', 'Unknown', '-', '-', '')
청크 1 - 10개의 데이터 삽입 완료 (총 10개)
모든 데이터 삽입 완료: 총 10개의 행
데이터베이스 연결 종료


In [22]:
import psycopg2

def select_all_products():
    """products 테이블의 모든 데이터를 조회합니다."""
    conn = None
    try:
        # DB 연결
        conn = get_db_connection()
        cur = conn.cursor()

        # 1. SELECT 쿼리 실행
        cur.execute("SELECT * FROM vessel WHERE id < 5 ORDER BY id;")

        # 2. 조회된 모든 데이터를 한 번에 가져오기
        products = cur.fetchall()

        # 3. 결과 출력
        print("--- 조회 상품 목록 ---")
        for product in products:
            print(product)

    except (Exception, psycopg2.DatabaseError) as error:
        print("오류 발생:", error)
    finally:
        if conn is not None:
            cur.close()
            conn.close()

select_all_products()

--- 조회 상품 목록 ---
(1, 'R000001', 'Containerships', 'Fully Cellular Container', 'Intermediate 3-7,999 TEU', '-', '4500', 'TEU', '55000', '47529', '28779.64955', '-', '-', '-', '-', '-', '2025-07-18 00:00:00', '2025', '2028-04-01 00:00:00', '2028', 'Taizhou Sanfu', 'China P.R.', 'Taizhou Sanfu', 'China P.R.', 'MPC Container Ships', 'Cyprus', 'Castor Maritime', 'Cyprus', '-', 'On Order', '57', '$m', 'Diesel 2-Stroke', 'VLS IFO', 'Diesel', 'Unknown', '1 x Diesel - Everllence B. & W. 6G70ME-C10.5 - 2-stroke 6-cyl. 700mm x3256mm bore/stroke 18,600mkW total at 78rpm', '3 x Aux. Diesel Gen. - 4-stroke  driving 3 x AC generator(s)', '1 x Emergency Diesel Gen. - 4-stroke  driving 1 x AC generator(s)', '-', '1 x Screw Shaft', '-', '3 x Selective Catalytic Reduction - NOx(Low Pressure) - Unknown, 1 x BWTS - Ballast Water Treatment System', '-', 'Unknown', '-', '-', '', datetime.datetime(2025, 10, 7, 0, 14, 25, 338756, tzinfo=datetime.timezone.utc))
(2, 'R000002', 'Containerships', 'Fully Cellular C

In [59]:
# import psycopg2

# def delete_product_by_id(product_id):
#     """특정 ID를 가진 상품을 products 테이블에서 삭제합니다."""
#     conn = None
#     deleted_rows = 0
#     try:
#         # DB 연결
#         conn = get_db_connection()
#         cur = conn.cursor()

#         # 1. DELETE 쿼리 실행
#         delete_query = "DELETE FROM builder WHERE id = %s;"
#         cur.execute(delete_query, (product_id,))

#         # 2. 삭제된 행 수 확인
#         deleted_rows = cur.rowcount

#         # 3. 변경사항 커밋
#         conn.commit()

#         if deleted_rows > 0:
#             print(f"ID {product_id}인 상품이 성공적으로 삭제되었습니다.")
#         else:
#             print(f"ID {product_id}인 상품을 찾을 수 없습니다.")

#     except (Exception, psycopg2.DatabaseError) as error:
#         print("오류 발생:", error)
#         # 오류 발생 시 롤백
#         if conn is not None:
#             conn.rollback()
#     finally:
#         if conn is not None:
#             cur.close()
#             conn.close()
    
#     return deleted_rows

# # 사용 예시
# if __name__ == "__main__":
#     # 삭제할 상품 ID 입력
#     product_id_to_delete = 2  # 여기에 삭제하려는 상품 ID를 입력하세요
    
#     # 상품 삭제 실행
#     delete_product_by_id(product_id_to_delete)
    
#     # 삭제 후 전체 상품 목록 확인 (선택사항)
#     select_all_products()